In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L3J1bi9tZWRpYS9hbGYvZGF0b3MvbWlzcmVwb3NpdG9yaW9zL2RvY2VuY2lhL2FwcmVuZGl6YWplLWF1dG9tYXRpY28tcHJhY3RpY2FzLWp1bGlh"
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using CSV, DataFrames
df = CSV.read("datos/tenis.csv", DataFrame)

Row,Cielo,Temperatura,Humedad,Viento,Tenis
,String15,String15,String7,String7,String3
1,Soleado,Caluroso,Alta,Suave,No
2,Soleado,Caluroso,Alta,Fuerte,No
3,Nublado,Caluroso,Alta,Suave,Sí
4,Lluvioso,Moderado,Alta,Suave,Sí
5,Lluvioso,Frío,Normal,Suave,Sí
6,Lluvioso,Frío,Normal,Fuerte,No
7,Nublado,Frío,Normal,Fuerte,Sí
8,Soleado,Moderado,Alta,Suave,No
9,Soleado,Frío,Normal,Suave,Sí


In [3]:
using GLMakie, AlgebraOfGraphics

function frecuencias(df::DataFrame, var::Symbol)
    frec = combine(groupby(df, [var, :Tenis]), nrow => :Días)
    plt = data(frec) * 
    mapping(var, :Días, stack = :Tenis, color = :Tenis, ) * 
    visual(BarPlot) 
    return plt
end

f = Figure()
draw!(f[1, 1], frecuencias(df, :Cielo))
draw!(f[1, 2], frecuencias(df, :Temperatura))
draw!(f[1, 3], frecuencias(df, :Humedad))
draw!(f[1, 4], frecuencias(df, :Viento))

1×1 Matrix{AxisEntries}:
 AxisEntries(Axis (1 plots), Entry[Entry(Plot{Makie.barplot}, Any[[2, 1, 2, 1], [2, 3, 6, 3]], {:direction = :y, :stack = [1, 1, 2, 2], :color = ColorTypes.RGBA{Float32}[RGBA(0.0, 0.44705883, 0.69803923, 1.0), RGBA(0.0, 0.44705883, 0.69803923, 1.0), RGBA(0.9019608, 0.62352943, 0.0, 1.0), RGBA(0.9019608, 0.62352943, 0.0, 1.0)], :cycle = nothing})], {AlgebraOfGraphics.AesStack = {nothing = AlgebraOfGraphics.CategoricalScale{Vector{String3}, Base.OneTo{Int64}}(String3["No", "Sí"], Base.OneTo(2), "Tenis", AlgebraOfGraphics.CategoricalScaleProps(AlgebraOfGraphics.EmptyCategoricalProps(), nothing, true, nothing, nothing), AlgebraOfGraphics.AesStack)}, AlgebraOfGraphics.AesColor = {nothing = AlgebraOfGraphics.CategoricalScale{Vector{String3}, Vector{ColorTypes.RGBA{Float32}}}(String3["No", "Sí"], ColorTypes.RGBA{Float32}[RGBA(0.0, 0.44705883, 0.69803923, 1.0), RGBA(0.9019608, 0.62352943, 0.0, 1.0)], "Tenis", AlgebraOfGraphics.CategoricalScaleProps(AlgebraOfGraphics.Em